# Problem 
- problem 다예 그림 넣기
- text-to-video retrieval task 정의
- emotion-specialized한 영상 추출
- input / output,우리는 text에서 emotion을 추출

- task 설명 
- Emotion-Specialized Text-to-Video Retrieval Task
  - 사용자가 input으로 추출하고자 하는 비디오 설명 text를 입력한다. inference 단계에서 cosine similarity matrix를 만들기 위해 우리의 emotion-specialized text-to-video retrieval model 가 관련 쿼리사용자의 디렉토리에 있는 비디오와 해당 쿼리 pair 간의 유사성을 계산한다. 
  -  inference 할 때에는, 데이터 속 text 쿼리에 기반한 embedding features와 cosine similarity가 가까운 video vectors를 similarity가 가장 큰 순서로 나열하여 가장 연관된 비디오 상위 n개를 추출하는 task이다. 
  - 우리는 텍스트와 비디오 두 modalities간의 연관성을 잘 배운 text-to-video / video-to-text retrieval model인 CLIP-ViP 모델을 이용하였다. 
  - 여기서 추가적인 우리의 목표는, 감정과 관련된 쿼리에 대해서 더 retrieval task를 잘하는 모델을 만드는 것이었다. 
  - 그 이유는, 사람은 추억 속에 살기 때문에 만약 특정 영상을 찾아야 한다면 감정 관련된 묘사를 하는 쿼리를 입력할 것이라는 예상을 하였기 때문이다.
  - 따라서 text에서 8개의 emotion 추출한 후 embedding을 하여 embedding space에서 각 감정에 대해 text와 video가 잘 clustering되도록 발전시킬 수 있을 것이라 생각하였다. 

![fig1](problem.png)


- baseline model (그림 추가 해야하나?)
  - 데이터 속 비디오와 해당 비디오의 caption을 같은 embedding space에 mapping 한다. 
  - contrastive learning을 통해 positive pair끼리 가깝도록 학습시킨다.
  - constrastive learning이란, self-supervised learning에서 positive pair간의 embedding feature vectors는 가까이,  negative pairs embedding feature vectors 들끼리는 멀리 배치하도록 하는 기법이다. 
  - baseline model의 training 시에는, (Video, video 속 각 frame) = (V, F) 과 (subtitle, caption) = (S, C) 로 각각 concat하여 joint Cross-Modal Learning on the omnisource input 을 한다. 
    - subtitle: video 전체를 아우르는 제목 
    - caption: video를 frame으로 나누었을 때, 각 이미지에 대한 caption
    - video: video 
    - frame: video를 12 frame으로 자름 
    - 즉, positive pair로는 (V, S), (V, C), (F, C) 의 다양한 조합을 이용하여 loss funtion을 구성한다. 
    - (논문) To learn rich video-language alignment from video-subtitle pairs and reduce the language domain gap with downstream data by corresponding auxiliary frame-caption pairs, we study joint Cross-Modal Learning on the omnisource input. 

# Loss function 

- 해당 a source-wise info-NCE loss by: 
$\mathcal{L}_{v 2 t}=-\frac{1}{B} \sum_{i=1}^B \log \frac{e^{v_i^{\top} t_i / \tau}}{\sum_{j=1}^B e^{v_i^{\top} t_j / \tau}}, \quad \mathcal{L}_{t 2 v}=-\frac{1}{B} \sum_{i=1}^B \log \frac{e^{t_i^{\top} v_i / \tau}}{\sum_{j=1}^B e^{t_i^{\top} v_j / \tau}}$